In [1]:
# ============================================================
# Simulated Catalog Pipeline (Colab-ready, CSV + XY range control)
# - No NaNs/Inf
# - Exact column order matching the provided schema
# - Adjustable resolution via pixscale & FOV (Mode A)
# - Or direct XY range control (Mode B)
# - Quick validation & preview
# ============================================================

from dataclasses import dataclass
from typing import Optional, List, Tuple
import numpy as np
import pandas as pd
from typing import Dict, Tuple
import math
from pathlib import Path
import os
import astropy.units as u
from astropy.cosmology import Planck18 as COSMO
from astropy.constants import G, c as c_light

In [2]:
def pixels_from_fov(fov_arcmin_x: float, fov_arcmin_y: float, pixscale_arcsec: float) -> Tuple[int,int]:
    w = int(round((fov_arcmin_x*60.0) / pixscale_arcsec))
    h = int(round((fov_arcmin_y*60.0) / pixscale_arcsec))
    return max(w, 10), max(h, 10)

@dataclass
class ClusterCFG:
    N_CLUSTERS: int = 2                 # 默认两个峰
    SEP_X: float = 2000.0               # 两峰在 x 的间距（默认单位：像素）
    SEP_UNITS: str = "pixels"           # 'pixels' 或 'arcsec'
    X_OFFSETS_PIX: Optional[List[float]] = None  # N>2时可显式给相对中心的像素偏移

@dataclass
class NFWParams:
    """物理版 NFW 参数，方便在后面注入 shear 时使用。"""
    M200: float          # [Msun]
    c200: float
    z_l: float           # lens redshift
    z_s: float           # source redshift
    rs_mpc: float        # NFW scale radius [Mpc]
    r200_mpc: float      # r_200 [Mpc]
    rs_arcsec: float     # scale radius in arcsec
    rs_pix: float        # scale radius in pixels (和 pixscale 相关)
    kappa_s: float       # NFW 强度参数 (dimensionless)


def nfw_params_from_M_c_z(
    M200: float,
    c200: float,
    z_l: float,
    z_s: float,
    pixscale_arcsec: float,
    cosmo=COSMO,
) -> NFWParams:
    """
    给定 M200, c200, lens/source redshift, 像素尺度，
    计算物理版 NFW 需要的 rs_arcsec, rs_pix, kappa_s。

    参数
    ----
    M200 : float
        halo 质量 M200，单位 Msun。
    c200 : float
        NFW 浓度参数 c = r200 / rs。
    z_l : float
        lens redshift (cluster redshift)。
    z_s : float
        source redshift (背景星系)，必须 z_s > z_l。
    pixscale_arcsec : float
        像素尺⼨，单位 arcsec/pix。
    cosmo : astropy.cosmology
        使用的宇宙学模型，默认 Planck18。

    返回
    ----
    NFWParams
        里面包含:
          - rs_mpc, r200_mpc
          - rs_arcsec, rs_pix
          - kappa_s
          - 以及原始输入 M200, c200, z_l, z_s
    """
    if z_s <= z_l:
        raise ValueError(f"z_s must be > z_l. Got z_l={z_l}, z_s={z_s}")

    # ---- 1. 质量/临界密度 -> r200 ----
    M200_phys = M200 * u.Msun

    # 临界密度 ρ_crit(z_l) [Msun / Mpc^3]
    rho_crit = cosmo.critical_density(z_l).to(u.Msun / u.Mpc**3)

    # r200：球内平均密度 = 200 * ρ_crit
    r200 = (3 * M200_phys / (4 * np.pi * 200 * rho_crit))**(1.0 / 3.0)  # [Mpc]
    r200 = r200.to(u.Mpc)

    # ---- 2. r200 + c200 -> rs ----
    rs = r200 / c200  # [Mpc]

    # ---- 3. 角径距 & Σ_crit ----
    D_l = cosmo.angular_diameter_distance(z_l)   # [Mpc]
    D_s = cosmo.angular_diameter_distance(z_s)   # [Mpc]
    D_ls = cosmo.angular_diameter_distance_z1z2(z_l, z_s)  # [Mpc]

    # critical surface density Σ_crit [Msun / Mpc^2]
    Sigma_crit = (c_light**2 / (4.0 * np.pi * G) * (D_s / (D_l * D_ls)))
    Sigma_crit = Sigma_crit.to(u.Msun / u.Mpc**2)

    # ---- 4. NFW density scale ρ_s & kappa_s ----
    c = float(c200)
    # δ_c: characteristic overdensity
    delta_c = (200.0 / 3.0) * c**3 / (np.log(1.0 + c) - c / (1.0 + c))

    rho_s = delta_c * rho_crit     # [Msun / Mpc^3]

    # 注意：kappa_s = ρ_s * r_s / Σ_crit
    kappa_s = (rho_s * rs / Sigma_crit).decompose().value  # dimensionless

    # ---- 5. rs 转成角尺度 rs_arcsec & 像素单位 rs_pix ----
    # rs 的角大小 θ_s = rs / D_l  [rad]
    theta_s_rad = (rs / D_l).to(u.dimensionless_unscaled).value  # 单位: rad

    # rad -> arcsec
    rs_arcsec = theta_s_rad * (180.0 / np.pi) * 3600.0  # [arcsec]

    # arcsec -> pixel
    rs_pix = rs_arcsec / pixscale_arcsec

    return NFWParams(
        M200=float(M200),
        c200=float(c200),
        z_l=float(z_l),
        z_s=float(z_s),
        rs_mpc=float(rs.value),
        r200_mpc=float(r200.value),
        rs_arcsec=float(rs_arcsec),
        rs_pix=float(rs_pix),
        kappa_s=float(kappa_s),
    )

def _sep_to_pixels(sep_value: float, units: str, pixscale_arcsec: float) -> float:
    if units.lower() == "pixels":
        return float(sep_value)
    elif units.lower() == "arcsec":
        return float(sep_value) / float(pixscale_arcsec)
    raise ValueError("SEP_UNITS 必须是 'pixels' 或 'arcsec'")

def compute_cluster_peaks(
    width_px: int,
    height_px: int,
    ra0: float,
    dec0: float,
    pixscale_arcsec: float,
    cfg,                   # 允许传 ClusterCFG 或简单对象，需有 N_CLUSTERS, SEP_X, SEP_UNITS
    save_csv: bool = False,
    csv_path: str = "peaks.csv"
):
    """
    支持 N_CLUSTERS = 0/1/2/…：
    - 0: 返回空 DataFrame（不保存CSV）
    - 1: 单峰放在 (x0, y0) 视场中心
    - >=2: 一个在中心（若奇数），其余按 SEP_X 沿 x 轴两侧对称展开；y 全在中线
    """
    
    n = int(getattr(cfg, "N_CLUSTERS", 2))
    sep_val = float(getattr(cfg, "SEP_X", 2000.0))
    units   = str(getattr(cfg, "SEP_UNITS", "pixels")).lower()

    # 像素中心
    x0 = width_px  / 2.0
    y0 = height_px / 2.0

    # 把间距转成像素
    if units == "pixels":
        sep_pix = sep_val
    elif units == "arcsec":
        sep_pix = sep_val / float(pixscale_arcsec)
    else:
        raise ValueError("SEP_UNITS must be 'pixels' or 'arcsec'")

    # 0 个 cluster 直接返回空
    if n <= 0:
        return pd.DataFrame(columns=["cluster_id","x_peak","y_peak","ra_peak","dec_peak"])

    # 生成 x 位置：奇数包含中心；两侧按 ±k*sep_pix 对称
    xs = []
    if n % 2 == 1:
        xs.append(x0)  # 中心
    k = 1
    while len(xs) < n:
        xs.extend([x0 - k*sep_pix, x0 + k*sep_pix])
        k += 1
    xs = xs[:n]
    xs = np.array(xs, dtype=float)

    ys = np.full(n, y0, dtype=float)

    # 小角度把像素转成 RA/Dec（只为写到 peaks.csv）
    deg_per_pix = pixscale_arcsec / 3600.0
    cosdec = math.cos(math.radians(dec0)) or 1e-9
    ra_peak  = ra0  + (xs - x0) * deg_per_pix / cosdec
    dec_peak = dec0 + (ys - y0) * deg_per_pix

    peaks = pd.DataFrame({
        "cluster_id": np.arange(1, n+1, dtype=int),
        "x_peak": xs, "y_peak": ys,
        "ra_peak": ra_peak, "dec_peak": dec_peak
    })

    if save_csv and n > 0:
        peaks.to_csv(csv_path, index=False, float_format="%.9f")
    return peaks

def annotate_nearest_cluster(
    sources: pd.DataFrame,
    peaks_df: pd.DataFrame,
    x_col: str = "x",
    y_col: str = "y",
) -> pd.DataFrame:
    """给源表打上最近峰：nearest_cluster_id（像素距离），以及 dist_pix。"""
    xp = peaks_df["x_peak"].to_numpy()
    yp = peaks_df["y_peak"].to_numpy()
    cid = peaks_df["cluster_id"].to_numpy()

    xs = sources[x_col].to_numpy()
    ys = sources[y_col].to_numpy()

    dx = xs[:, None] - xp[None, :]
    dy = ys[:, None] - yp[None, :]
    d2 = dx*dx + dy*dy
    idx_min = np.argmin(d2, axis=1)
    nearest = cid[idx_min]
    dist = np.sqrt(d2[np.arange(len(xs)), idx_min])

    out = sources.copy()
    out["nearest_cluster_id"] = nearest
    out["dist_pix"] = dist
    return out

def inject_shear_SIS(
    df: pd.DataFrame,
    peaks_df: pd.DataFrame,
    theta_E_arcsec: float = 20.0,   # 爱因斯坦角（角秒）— 决定信号强弱；20~40 比较显眼
    core_arcsec: float = 2.0,       # 核心软化（角秒）防止中心发散
    max_gt: float = 0.4,            # 剪裁最大切向 shear，避免不合理的大形变
    x_col: str = "x",
    y_col: str = "y",
    e1_col: str = "e1",
    e2_col: str = "e2",
    write_to_new_cols: bool = True  # True: 写 e1_sheared/e2_sheared；False: 覆盖 e1/e2
) -> pd.DataFrame:
    xs = df[x_col].to_numpy()
    ys = df[y_col].to_numpy()
    e1 = df[e1_col].to_numpy().copy()
    e2 = df[e2_col].to_numpy().copy()

    # 角尺度换算
    pix2arc = CFG.PIXSCALE_ARCSEC  # arcsec per pixel

    g1_add = np.zeros_like(e1)
    g2_add = np.zeros_like(e2)

    for _, pk in peaks_df.iterrows():
        dx = xs - pk["x_peak"]
        dy = ys - pk["y_peak"]
        R_pix = np.sqrt(dx*dx + dy*dy)
        R_arc = np.sqrt((R_pix*pix2arc)**2 + core_arcsec**2)  # 带 core 的半径（角秒）

        gt = theta_E_arcsec / (2.0 * R_arc)  # SIS 切向 shear
        gt = np.clip(gt, 0.0, max_gt)

        phi = np.arctan2(dy, dx)
        g1_add += -gt * np.cos(2.0*phi)
        g2_add += -gt * np.sin(2.0*phi)

    if write_to_new_cols:
        df = df.copy()
        df["e1_sheared"] = e1 + g1_add
        df["e2_sheared"] = e2 + g2_add
    else:
        df = df.copy()
        df[e1_col] = e1 + g1_add
        df[e2_col] = e2 + g2_add

    # 安全兜底：无非有限
    for c in ["e1_sheared","e2_sheared"] if write_to_new_cols else [e1_col,e2_col]:
        arr = df[c].to_numpy()
        mask = ~np.isfinite(arr)
        if mask.any():
            arr[mask] = 0.0
            df[c] = arr
    return df


# ==========================
# 物理版 NFW lensing 核心公式
# 参考 Bartelmann (1996), Wright & Brainerd (2000)
# ==========================

def _nfw_F(x):
    """
    NFW lensing 中用到的辅助函数 F(x)，x = R / r_s。
    """
    x = np.asarray(x, dtype=float)
    out = np.zeros_like(x)

    # x < 1
    m1 = x < 0.999
    if np.any(m1):
        xm = x[m1]
        a = np.sqrt((1.0 - xm) / (1.0 + xm))
        out[m1] = 0.5 * np.log((1.0 + a) / (1.0 - a)) / np.sqrt(1.0 - xm**2)

    # x > 1
    m2 = x > 1.001
    if np.any(m2):
        xm = x[m2]
        a = np.sqrt((xm - 1.0) / (xm + 1.0))
        out[m2] = np.arctan(a) / np.sqrt(xm**2 - 1.0)

    # x ~ 1 用近似展开，避免数值不稳定
    m3 = (x >= 0.999) & (x <= 1.001)
    if np.any(m3):
        xm = x[m3]
        out[m3] = 5.0/6.0 - xm/3.0   # 这一近似精度已经足够模拟用

    return out


def nfw_kappa_x(x, kappa_s):
    """
    NFW 的收敛度 κ(x)，x = R / r_s (无量纲), kappa_s 是强度参数。
    这里用了标准解析式：
        κ(x) = 2 kappa_s * f(x)
    """
    x = np.asarray(x, dtype=float)
    kappa_s = float(kappa_s)
    out = np.zeros_like(x)

    # x < 1
    m1 = x < 0.999
    if np.any(m1):
        xm = x[m1]
        a = np.sqrt((1.0 - xm) / (1.0 + xm))
        out[m1] = 2.0 * kappa_s * (
            1.0 / (xm**2 - 1.0) * (
                1.0 - np.log((1.0 + a) / (1.0 - a)) / np.sqrt(1.0 - xm**2)
            )
        )

    # x > 1
    m2 = x > 1.001
    if np.any(m2):
        xm = x[m2]
        a = np.sqrt((xm - 1.0) / (xm + 1.0))
        out[m2] = 2.0 * kappa_s * (
            1.0 / (xm**2 - 1.0) * (
                1.0 - 2.0 * np.arctan(a) / np.sqrt(xm**2 - 1.0)
            )
        )

    # x ~ 1
    m3 = (x >= 0.999) & (x <= 1.001)
    if np.any(m3):
        xm = x[m3]
        out[m3] = kappa_s * (22.0/15.0 - 0.8 * xm)  # 连续光滑的近似

    return out


def nfw_gamma_t_x(x, kappa_s):
    """
    NFW 的切向 shear γ_t(x)，x = R / r_s。
    解析式：
        γ_t(x) = 4 kappa_s [ ln(x/2) + F(x) ] / x^2 - κ(x)
    """
    x = np.asarray(x, dtype=float)
    kappa_s = float(kappa_s)
    out = np.zeros_like(x)

    # 避免 R=0
    m = x > 1e-4
    if np.any(m):
        xm = x[m]
        out[m] = (
            4.0 * kappa_s * (np.log(xm / 2.0) + _nfw_F(xm)) / (xm**2)
            - nfw_kappa_x(xm, kappa_s)
        )

    # x → 0 时的近似展开
    m0 = ~m
    if np.any(m0):
        xm = x[m0]
        # 简单的有限值近似，实际差异只影响极少数中心像素
        out[m0] = 4.0 * kappa_s * 0.25

    return out

def inject_shear_NFW_strict(
    df: pd.DataFrame,
    peaks_df: pd.DataFrame,
    pixscale_arcsec: float,
    rs_arcsec: float = 60.0,
    kappa_s: float = 0.2,
    e1_col: str = "e1",
    e2_col: str = "e2",
    write_to_new_cols: bool = False,
    clip_e: float = 0.99,
) -> pd.DataFrame:
    """
    用物理版 NFW 模型给 catalog 中的星系注入 shear，
    使用严格的椭率变换公式:
        e_obs = (e_int + g) / (1 + g* conj(e_int))

    参数：
    --------
    df : DataFrame
        catalog，必须有列 x, y, e1, e2（列名可通过 e1_col/e2_col 改）
    peaks_df : DataFrame
        每一行对应一个 cluster，至少要有 x_peak, y_peak 列（像素单位）
    pixscale_arcsec : float
        像素尺度 [arcsec / pix]
    rs_arcsec : float
        NFW 的 scale radius [arcsec]，所有 cluster 共用（简单版本）
    kappa_s : float
        NFW 的强度参数（等价于质量/Σ_crit 的组合），所有 cluster 共用
    write_to_new_cols : bool
        如果 True，则写入 e1_sheared, e2_sheared；否则直接覆盖 e1,e2。
    clip_e : float
        限制 |e| < clip_e，避免极端像素数值爆炸。
    """
    df = df.copy()

    x_gal = df["x"].to_numpy()
    y_gal = df["y"].to_numpy()

    # 初始 e_int
    e1_int = df[e1_col].to_numpy().astype(float)
    e2_int = df[e2_col].to_numpy().astype(float)
    e_int = e1_int + 1j * e2_int

    # 累加所有 halo 对每个星系的 κ 和 γ1/γ2
    kappa_tot = np.zeros_like(e1_int)
    gamma1_tot = np.zeros_like(e1_int)
    gamma2_tot = np.zeros_like(e1_int)

    rs_arcsec = float(rs_arcsec)
    pix2arc = float(pixscale_arcsec)

    for _, pk in peaks_df.iterrows():
        xc = pk["x_peak"]
        yc = pk["y_peak"]

        dx = x_gal - xc
        dy = y_gal - yc

        R_pix = np.sqrt(dx*dx + dy*dy)
        R_arc = R_pix * pix2arc     # 转成 arcsec
        x_dimless = R_arc / rs_arcsec   # x = R / r_s

        # 本 halo 的 κ 和 γ_t
        kappa_h = nfw_kappa_x(x_dimless, kappa_s)
        gamma_t = nfw_gamma_t_x(x_dimless, kappa_s)

        # 切向 shear → (γ1, γ2)
        phi = np.arctan2(dy, dx)
        cos2phi = np.cos(2.0 * phi)
        sin2phi = np.sin(2.0 * phi)

        gamma1_h = -gamma_t * cos2phi
        gamma2_h = -gamma_t * sin2phi

        kappa_tot += kappa_h
        gamma1_tot += gamma1_h
        gamma2_tot += gamma2_h

    # 计算总的 reduced shear g = γ_tot / (1 - κ_tot)
    denom = 1.0 - kappa_tot
    # 为避免奇异，做个保护
    small = np.abs(denom) < 1e-3
    denom[small] = 1e-3 * np.sign(denom[small] + 1e-6)

    g1 = gamma1_tot / denom
    g2 = gamma2_tot / denom
    g = g1 + 1j * g2

    # 椭率严格变换公式： e_obs = (e_int + g) / (1 + g* conj(e_int))
    g_conj = np.conj(g)
    e_int_conj = np.conj(e_int)
    num = e_int + g
    den = 1.0 + g_conj * e_int

    # 避免除零
    small_den = np.abs(den) < 1e-6
    den[small_den] = 1e-6 * np.sign(den[small_den] + 1e-6)

    e_obs = num / den

    # 限制 |e|，避免极端像素数值
    abs_e = np.abs(e_obs)
    too_big = abs_e > clip_e
    if np.any(too_big):
        e_obs[too_big] *= (clip_e / abs_e[too_big])

    e1_obs = np.real(e_obs)
    e2_obs = np.imag(e_obs)

    if write_to_new_cols:
        df["e1_sheared"] = e1_obs
        df["e2_sheared"] = e2_obs
    else:
        df[e1_col] = e1_obs
        df[e2_col] = e2_obs

    return df

In [ ]:
# -------------------------------
# 1) Hyper-parameters (edit here)
# -------------------------------
class CFG:
    # ---------- Common ----------
    # Field center (deg) — used to anchor RA/Dec projection
    RA0 = 44.86
    DEC0 = 11.26

    # Pixel scale (arcsec/pixel) — used for RA/Dec <-> pixel angle conversion
    PIXSCALE_ARCSEC = 0.263

    # Seeing (arcsec) — used for size-ish heuristics, not PSF convolution
    SEEING_FWHM_ARCSEC = 0.9

    # Population
    N_OBJ = 500000
    FRAC_EXTENDED = 0.7  # galaxy fraction
    SEED = 2025

    # NFW 物理参数
    M200 = 8e14        # Msun，比如 8×10^14 Msun
    C200 = 4.0         # 浓度参数
    Z_L  = 0.3         # lens redshift
    Z_S  = 0.8         # source redshift

    # 每个 sep 生成多少个随机 realization
    N_REAL_PER_SEP = 10

    # ---------- Cluster separation (统一入口) ----------
    SEP_LIST = list(range(1000, 10001, 1000))  # 这里放多个间距
    SEP_UNITS = "pixels"                    # 或 "arcsec"

    # ---------- Output ----------
    OUT_DIR = "batch_sep"                   # 统一放到目录里
    PREVIEW_N = 5

    # ---------- Mode select ----------
    # Mode A: use FOV (arcmin) + pixscale to set XY extent
    # Mode B: directly set XY min/max
    USE_XY_RANGE = False  # set True to enable Mode B

    # --- Mode A params (FOV-based) ---
    FOV_ARCMIN_X = 180.0
    FOV_ARCMIN_Y = 180.0

    # --- Mode B params (explicit XY range) ---
    X_MIN = 26000.0
    X_MAX = 28000.0
    Y_MIN = 2580.0
    Y_MAX = 2660.0

width_px, height_px = pixels_from_fov(
    CFG.FOV_ARCMIN_X, CFG.FOV_ARCMIN_Y, CFG.PIXSCALE_ARCSEC
)

# ---------------------------
# 2) Column schema & helpers
# ---------------------------
COLUMNS = [
    "ra","dec","x","y","e1","e2","res","sigmae","rkron","extendedness","blendedness","psf_used",
    "e1_sdss","e2_sdss","e1_psf_sdss","e2_psf_sdss","e1_hsm","e2_hsm","e1_psf_hsm","e2_psf_hsm",
    "i_e1_psf_sdss","i_e2_psf_sdss",
    "u_psf_mag","u_psf_magerr","u_cmodel_mag","u_cmodel_magerr",
    "g_psf_mag","g_psf_magerr","g_cmodel_mag","g_cmodel_magerr",
    "r_psf_mag","r_psf_magerr","r_cmodel_mag","r_cmodel_magerr",
    "i_psf_mag","i_psf_magerr","i_cmodel_mag","i_cmodel_magerr",
    "z_psf_mag","z_psf_magerr","z_cmodel_mag","z_cmodel_magerr",
    "Y_psf_mag","Y_psf_magerr","Y_cmodel_mag","Y_cmodel_magerr",
    "idn"
]

def magerr_from_mag(mag, m0=22.0, base_err=0.02, min_err=0.005, max_err=1.5):
    err = base_err * (10.0 ** (0.4 * (mag - m0)))
    return np.clip(err, min_err, max_err)

def draw_mags(n, band_offset=0.0, rng=None) -> Tuple[np.ndarray, np.ndarray]:
    if rng is None:
        rng = np.random.default_rng()
    mag = rng.normal(loc=23.0 + band_offset, scale=1.2, size=n)
    mag = np.clip(mag, 18.0, 27.5)
    cmodel = mag - rng.normal(loc=0.1, scale=0.07, size=n)
    return mag, cmodel

def gaussian_ellipticities(n, sigma=0.8, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    e1 = rng.normal(0.0, sigma, size=n)
    e2 = rng.normal(0.0, sigma, size=n)
    e = np.sqrt(e1**2 + e2**2)
    mask = e > 5
    if np.any(mask):
        e1[mask] *= 5 / e[mask]
        e2[mask] *= 5 / e[mask]
    return e1, e2

# -------------------------------------
# 3) Core simulation following the spec
# -------------------------------------
def simulate_catalog(
    n_obj:int,
    ra0:float, dec0:float,
    pixscale_arcsec:float,
    seeing_fwhm_arcsec:float,
    frac_extended:float,
    seed:int,
    use_xy_range:bool,
    fov_arcmin_x:float=None, fov_arcmin_y:float=None,
    x_min:float=None, x_max:float=None,
    y_min:float=None, y_max:float=None
) -> pd.DataFrame:

    rng = np.random.default_rng(seed)

    if use_xy_range:
        # -------- Mode B: explicit XY range --------
        if not (x_min is not None and x_max is not None and y_min is not None and y_max is not None):
            raise ValueError("USE_XY_RANGE=True 需要设置 X_MIN/X_MAX/Y_MIN/Y_MAX")
        if not (x_max > x_min and y_max > y_min):
            raise ValueError("X_MAX 必须 > X_MIN 且 Y_MAX 必须 > Y_MIN")

        # sample in given range
        x = rng.uniform(x_min, x_max, size=n_obj)
        y = rng.uniform(y_min, y_max, size=n_obj)

        # define a 'center' for sky-projection
        x0 = 0.5*(x_min + x_max)
        y0 = 0.5*(y_min + y_max)

    else:
        # -------- Mode A: FOV + pixscale --------
        if fov_arcmin_x is None or fov_arcmin_y is None:
            raise ValueError("FOV 模式需要 FOV_ARCMIN_X/Y")
        width_px  = int(round((fov_arcmin_x*60.0) / pixscale_arcsec))
        height_px = int(round((fov_arcmin_y*60.0) / pixscale_arcsec))
        width_px  = max(width_px, 10)
        height_px = max(height_px, 10)

        x = rng.uniform(0, width_px, size=n_obj)
        y = rng.uniform(0, height_px, size=n_obj)

        x0 = width_px / 2.0
        y0 = height_px / 2.0

    # RA/Dec small-angle projection
    deg_per_pix = pixscale_arcsec / 3600.0
    cosdec = math.cos(math.radians(dec0)) or 1e-9
    ra  = ra0  + (x - x0) * deg_per_pix / cosdec
    dec = dec0 + (y - y0) * deg_per_pix

    # Labels & sizes
    is_extended = (rng.random(n_obj) < frac_extended).astype(int)
    extendedness = is_extended.copy()
    blendedness = rng.binomial(1, p=0.10, size=n_obj).astype(int)
    psf_used    = rng.binomial(1, p=0.20, size=n_obj).astype(int)

    rkron = rng.lognormal(mean=np.log(2.0), sigma=0.4, size=n_obj)
    rkron[is_extended==0] = rng.lognormal(mean=np.log(0.8), sigma=0.3, size=(is_extended==0).sum())

    # Bands
    band_offsets = dict(u=0.6, g=0.2, r=0.0, i=-0.1, z=-0.15, Y=-0.2)
    mags: Dict[str, np.ndarray] = {}
    magerrs: Dict[str, np.ndarray] = {}

    for b in ["u","g","r","i","z","Y"]:
        psf_mag, cmod_mag = draw_mags(n_obj, band_offset=band_offsets[b], rng=rng)
        cmod_mag = np.where(is_extended==1, cmod_mag, psf_mag - rng.normal(0.02, 0.01, size=n_obj))
        mags[f"{b}_psf_mag"]      = psf_mag
        mags[f"{b}_cmodel_mag"]   = cmod_mag
        magerrs[f"{b}_psf_magerr"]    = magerr_from_mag(psf_mag)
        magerrs[f"{b}_cmodel_magerr"] = magerr_from_mag(cmod_mag)

    # Shapes
    e1, e2 = gaussian_ellipticities(n_obj, sigma=0.3, rng=rng)
    def jitter(arr, scale): return arr + rng.normal(0.0, scale, size=n_obj)

    e1_sdss = jitter(e1, 0.03)
    e2_sdss = jitter(e2, 0.03)
    e1_psf_sdss = jitter(0.2*e1, 0.02)
    e2_psf_sdss = jitter(0.2*e2, 0.02)

    e1_hsm = jitter(e1, 0.02)
    e2_hsm = jitter(e2, 0.02)
    e1_psf_hsm = jitter(0.18*e1, 0.02)
    e2_psf_hsm = jitter(0.18*e2, 0.02)

    i_e1_psf_sdss = jitter(0.2*e1, 0.02)
    i_e2_psf_sdss = jitter(0.2*e2, 0.02)

    # res & sigmae from r-band brightness (SNR proxy)
    snr_proxy = 10.0 * (10.0 ** (-0.4*(mags["r_psf_mag"] - 21.0)))
    res     = np.clip(snr_proxy / (snr_proxy + 20.0), 0.0, 1.0)
    sigmae  = np.clip(0.4 / np.sqrt(snr_proxy + 1.0), 0.01, 0.6)

    idn = np.arange(1, n_obj+1, dtype=int)

    df = pd.DataFrame({
        "ra": ra, "dec": dec, "x": x, "y": y,
        "e1": e1, "e2": e2, "res": res, "sigmae": sigmae,
        "rkron": rkron,
        "extendedness": extendedness, "blendedness": blendedness, "psf_used": psf_used,
        "e1_sdss": e1_sdss, "e2_sdss": e2_sdss, "e1_psf_sdss": e1_psf_sdss, "e2_psf_sdss": e2_psf_sdss,
        "e1_hsm": e1_hsm, "e2_hsm": e2_hsm, "e1_psf_hsm": e1_psf_hsm, "e2_psf_hsm": e2_psf_hsm,
        "i_e1_psf_sdss": i_e1_psf_sdss, "i_e2_psf_sdss": i_e2_psf_sdss,
        "u_psf_mag": mags["u_psf_mag"], "u_psf_magerr": magerrs["u_psf_magerr"],
        "u_cmodel_mag": mags["u_cmodel_mag"], "u_cmodel_magerr": magerrs["u_cmodel_magerr"],
        "g_psf_mag": mags["g_psf_mag"], "g_psf_magerr": magerrs["g_psf_magerr"],
        "g_cmodel_mag": mags["g_cmodel_mag"], "g_cmodel_magerr": magerrs["g_cmodel_magerr"],
        "r_psf_mag": mags["r_psf_mag"], "r_psf_magerr": magerrs["r_psf_magerr"],
        "r_cmodel_mag": mags["r_cmodel_mag"], "r_cmodel_magerr": magerrs["r_cmodel_magerr"],
        "i_psf_mag": mags["i_psf_mag"], "i_psf_magerr": magerrs["i_psf_magerr"],
        "i_cmodel_mag": mags["i_cmodel_mag"], "i_cmodel_magerr": magerrs["i_cmodel_magerr"],
        "z_psf_mag": mags["z_psf_mag"], "z_psf_magerr": magerrs["z_psf_magerr"],
        "z_cmodel_mag": mags["z_cmodel_mag"], "z_cmodel_magerr": magerrs["z_cmodel_magerr"],
        "Y_psf_mag": mags["Y_psf_mag"], "Y_psf_magerr": magerrs["Y_psf_magerr"],
        "Y_cmodel_mag": mags["Y_cmodel_mag"], "Y_cmodel_magerr": magerrs["Y_cmodel_magerr"],
        "idn": idn
    })[COLUMNS]

    # Dtypes & sanitization
    for col in ["extendedness","blendedness","psf_used","idn"]:
        df[col] = df[col].astype(int)
    for col in df.columns:
        arr = df[col].to_numpy()
        if col in ["extendedness","blendedness","psf_used","idn"]:
            mask = ~np.isfinite(arr.astype(float))
            if mask.any(): arr[mask] = 0
            df[col] = arr.astype(int)
        else:
            mask = ~np.isfinite(arr)
            if mask.any(): arr[mask] = 0.0
            df[col] = arr
    return df

# ----------------------------
# 4) Validation & Quick Look
# ----------------------------
def validate_catalog(df: pd.DataFrame):
    assert list(df.columns) == COLUMNS, "Column order mismatch."
    bad = ~np.isfinite(df.to_numpy(dtype=float, copy=True))
    if bad.any():
        bad_cols = df.columns[np.unique(np.where(bad)[1])]
        raise ValueError(f"Non-finite values found in columns: {list(bad_cols)}")
    assert (df["res"].between(0,1)).all(), "`res` out of [0,1]"
    assert (df["sigmae"] > 0).all(), "`sigmae` must be positive"
    assert (df["idn"].iloc[0] == 1) and (np.diff(df["idn"]).min() >= 0), "idn not ascending from 1"

def quick_stats(df: pd.DataFrame):
    return {
        "n": len(df),
        "x_range": (df["x"].min(), df["x"].max()),
        "y_range": (df["y"].min(), df["y"].max()),
        "ra_range": (df["ra"].min(), df["ra"].max()),
        "dec_range": (df["dec"].min(), df["dec"].max()),
        "extended_frac": float(df["extendedness"].mean()),
        "blended_frac":  float(df["blendedness"].mean()),
        "psf_used_frac": float(df["psf_used"].mean()),
        "res_mean_std":  (df["res"].mean(), df["res"].std()),
        "sigmae_mean_std": (df["sigmae"].mean(), df["sigmae"].std()),
    }

def save_csv(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False, float_format="%.9f")  # CSV (逗号分隔)

# ----------------------------
# 5) 主脚本：多 sep × 多 realization
# ----------------------------

out_dir = Path(CFG.OUT_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

# 5.1) 先算好视场对应的像素宽高（后面要传给 compute_cluster_peaks）
width_px, height_px = pixels_from_fov(
    CFG.FOV_ARCMIN_X, CFG.FOV_ARCMIN_Y, CFG.PIXSCALE_ARCSEC
)
print(f"FOV -> width_px={width_px}, height_px={height_px}")

# 小工具：把 sep 变成文件名后缀，比如 2000 -> '2000px'
def _suffix(sep):
    return f"{int(sep)}px" if CFG.SEP_UNITS.lower() == "pixels" else f"{float(sep):g}arcsec"

for sep in CFG.SEP_LIST:
    print(f"\n=== SEP = {sep} ({CFG.SEP_UNITS}) ===")

    # 5.2) 对于每个 sep，先生成一次真值 cluster 位置（peaks）
    cfg_peaks = ClusterCFG(
        N_CLUSTERS=2,
        SEP_X=sep,
        SEP_UNITS=CFG.SEP_UNITS,
    )
    peaks_df = compute_cluster_peaks(
        width_px, height_px,
        CFG.RA0, CFG.DEC0, CFG.PIXSCALE_ARCSEC,
        cfg=cfg_peaks
    )

    # 保存 peaks（真值 cluster 坐标，仅依赖 sep，不依赖 realization）
    peaks_path = out_dir / f"sep{_suffix(sep)}_peaks.csv"
    peaks_df.to_csv(peaks_path, index=False, float_format="%.9f")

    # 5.3) 在同一个 sep 下，做多个随机 realization
    for i_real in range(1, CFG.N_REAL_PER_SEP + 1):
        # 给每个 realization 一个不同的随机种子
        seed = CFG.SEED + i_real + int(sep)*1000

        print(f"  - realization {i_real}/{CFG.N_REAL_PER_SEP}, seed={seed}")

        nfw_par = nfw_params_from_M_c_z(
            M200=CFG.M200,
            c200=CFG.C200,
            z_l=CFG.Z_L,
            z_s=CFG.Z_S,
            pixscale_arcsec=CFG.PIXSCALE_ARCSEC,
        )

        print("NFW parameters from (M200, c, z):")
        print(f"  r200   = {nfw_par.r200_mpc:.3f} Mpc")
        print(f"  rs     = {nfw_par.rs_mpc:.3f} Mpc")
        print(f"  rs_arcsec = {nfw_par.rs_arcsec:.2f} arcsec")
        print(f"  rs_pix    = {nfw_par.rs_pix:.1f} pix")
        print(f"  kappa_s   = {nfw_par.kappa_s:.3f}")

        # (a) 生成一个新的模拟 catalog（baseline，无 shear）
        df = simulate_catalog(
            n_obj=CFG.N_OBJ,
            ra0=CFG.RA0,
            dec0=CFG.DEC0,
            pixscale_arcsec=CFG.PIXSCALE_ARCSEC,
            seeing_fwhm_arcsec=CFG.SEEING_FWHM_ARCSEC,
            frac_extended=CFG.FRAC_EXTENDED,
            seed=seed,
            use_xy_range=CFG.USE_XY_RANGE,
            fov_arcmin_x=CFG.FOV_ARCMIN_X if not CFG.USE_XY_RANGE else None,
            fov_arcmin_y=CFG.FOV_ARCMIN_Y if not CFG.USE_XY_RANGE else None,
            x_min=CFG.X_MIN if CFG.USE_XY_RANGE else None,
            x_max=CFG.X_MAX if CFG.USE_XY_RANGE else None,
            y_min=CFG.Y_MIN if CFG.USE_XY_RANGE else None,
            y_max=CFG.Y_MAX if CFG.USE_XY_RANGE else None
        )

        validate_catalog(df)

        # (b) 按当前 sep 的 peaks 注入剪切（这里先保留 SIS，下一节换成 NFW）
        """
        df_out = inject_shear_SIS(
            df, peaks_df,
            theta_E_arcsec=30.0,
            core_arcsec=2.0,
            max_gt=0.4,
            write_to_new_cols=False   # 覆盖 e1/e2
        )
        """
        # NFW 注入剪切
        # NFW 参数（你可以调）
        df_out = inject_shear_NFW_strict(
            df,
            peaks_df,
            pixscale_arcsec=CFG.PIXSCALE_ARCSEC,
            rs_arcsec=nfw_par.rs_arcsec,
            kappa_s=nfw_par.kappa_s,
            e1_col="e1",
            e2_col="e2",
            write_to_new_cols=False,
        )

        validate_catalog(df_out)

        # (c) 保存 catalog，文件名带上 realization 编号
        cat_path = out_dir / f"sep{_suffix(sep)}.r{i_real:03d}.csv"
        df_out.to_csv(cat_path, index=False, float_format="%.9f")

        print(f"    [OK] -> {cat_path.name}")

print("All done.")


FOV -> width_px=41065, height_px=41065

=== SEP = 1000 (pixels) ===
  - realization 1/10, seed=1002026
NFW parameters from (M200, c, z):
  r200   = 1.763 Mpc
  rs     = 0.441 Mpc
  rs_arcsec = 95.85 arcsec
  rs_pix    = 364.5 pix
  kappa_s   = 0.132
    [OK] -> sep1000px.r001.csv
  - realization 2/10, seed=1002027
NFW parameters from (M200, c, z):
  r200   = 1.763 Mpc
  rs     = 0.441 Mpc
  rs_arcsec = 95.85 arcsec
  rs_pix    = 364.5 pix
  kappa_s   = 0.132
    [OK] -> sep1000px.r002.csv
  - realization 3/10, seed=1002028
NFW parameters from (M200, c, z):
  r200   = 1.763 Mpc
  rs     = 0.441 Mpc
  rs_arcsec = 95.85 arcsec
  rs_pix    = 364.5 pix
  kappa_s   = 0.132
    [OK] -> sep1000px.r003.csv
  - realization 4/10, seed=1002029
NFW parameters from (M200, c, z):
  r200   = 1.763 Mpc
  rs     = 0.441 Mpc
  rs_arcsec = 95.85 arcsec
  rs_pix    = 364.5 pix
  kappa_s   = 0.132
    [OK] -> sep1000px.r004.csv
  - realization 5/10, seed=1002030
NFW parameters from (M200, c, z):
  r200   = 